In [1]:
!pip install tensorflow_docs

In [23]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, Softmax
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import InceptionV3, EfficientNetB0, ResNet50, VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_input_inception
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_input_efficientnet
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots

In [24]:
# Function to load and preprocess images based on the chosen model
def preprocess_images(X, model_name):
    if model_name == 'InceptionV3':
        return preprocess_input_inception(X)
    elif model_name == 'EfficientNet':
        return preprocess_input_efficientnet(X)
    elif model_name == 'ResNet':
        return preprocess_input_resnet(X)
    elif model_name == 'VGG16':
        return preprocess_input_vgg(X)
    else:
        raise ValueError(f"Unsupported model: {model_name}")

# Function to build a specified pre-trained model
def build_pretrained_model(model_name, input_shape, num_classes):
    if model_name == 'EfficientNet':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'ResNet':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError(f"Unsupported model: {model_name}")

    # Add custom dense layers on top of the pre-trained base
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(63, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Function to plot the model history
def plot_model_history(model_history, metric, plot_name):
    sns.set(style='darkgrid')  # Use Seaborn directly to set the style
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)
    plt.title(f'{metric.upper()}')
    plt.ylim([0, 1])
    plt.savefig(f'{plot_name}.png')
    plt.close()

# Function to load and preprocess image paths
def load_image_paths(drive_folder_path, image_types=('*.jpg', '*.png', '*.jpeg', '*.gif')):
    image_paths = []
    classes = []

    # Get the list of subfolders inside the main folder
    subfolders = [f.path for f in os.scandir(drive_folder_path) if f.is_dir()]

    for folder in subfolders:
        class_name = os.path.basename(folder)  # Get the name of the folder as the class
        class_images = []

        # Iterate over specified image types
        for image_type in image_types:
            class_images.extend(glob(os.path.join(folder, image_type)))

        image_paths.extend(class_images)
        classes.extend([class_name] * len(class_images))

    return image_paths, classes

# Function to load images and labels
def load_images_and_labels(image_paths, target_size=(256,256)):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=target_size)
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]
        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

In [25]:
# Main function for training and evaluating pre-trained models
def main():
    SEED = 999
    np.random.seed(SEED)

    # Load the dataset into memory, normalizing the images and one-hot encoding the labels
    drive_folder_path = '/content/drive/MyDrive/Archaeological_Sites_Classification'  # Update with your actual folder path
    image_paths, classes = load_image_paths(drive_folder_path)
    X, y = load_images_and_labels(image_paths)
    X = X.astype('float') / 255.0
    label_binarizer = LabelBinarizer()
    y = label_binarizer.fit_transform(y)

    # Display class distribution
    class_df = pd.DataFrame({'Class': classes})
    print(class_df['Class'].value_counts())

    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.33, random_state=SEED)
    print(f'Shape of X_train: {X_train.shape}')
    print(f'Shape of X_test: {X_test.shape}')
    print(f'Shape of y_train: {y_train.shape}')
    print(f'Shape of y_test: {y_test.shape}')

    EPOCHS = 10
    BATCH_SIZE = 6

    # List of pre-trained models to evaluate
    pretrained_models = ['EfficientNet', 'InceptionV3', 'ResNet', 'VGG16']

    for model_name in pretrained_models:
        print(f"\nTraining and evaluating {model_name}...\n")

        # Build and compile the pre-trained model
        model = build_pretrained_model(model_name, (256, 256, 3), len(label_binarizer.classes_))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Apply data augmentation during training
        if model_name == 'EfficientNet':
            train_datagen = ImageDataGenerator(
                preprocessing_function=preprocess_input_efficientnet,
                horizontal_flip=True,
                rotation_range=30,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.2,
                zoom_range=0.2,
                fill_mode='nearest'
            )
        else:
            train_datagen = ImageDataGenerator(
                preprocessing_function=lambda x: preprocess_images(x, model_name),
                horizontal_flip=True,
                rotation_range=30,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.2,
                zoom_range=0.2,
                fill_mode='nearest'
            )

        train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

        # Train the model with data augmentation
        history = model.fit(train_generator,steps_per_epoch=len(X_train) // BATCH_SIZE, validation_data=(X_test, y_test), epochs=EPOCHS)

        # Evaluate the model
        result = model.evaluate(X_test, y_test)
        print(f'Test accuracy ({model_name}): {result[1]}')

        # Print classification report
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true_classes = np.argmax(y_test, axis=1)
        print(f'\nClassification Report ({model_name}):\n')
        print(classification_report(y_true_classes, y_pred_classes, target_names=label_binarizer.classes_))

        # Plot learning curve
        plot_model_history(history, 'accuracy', f'{model_name}_learning_curve')

# run script
if __name__ == "__main__":
    main()

um_qais          811
jarash           703
Wadi_Rum         614
Petra            607
Ajloun           572
Roman_Theater    520
Name: Class, dtype: int64
Shape of X_train: (3061, 256, 256, 3)
Shape of X_test: (766, 256, 256, 3)
Shape of y_train: (3061, 6)
Shape of y_test: (766, 6)

Training and evaluating EfficientNet...

Epoch 1/10
511/511 [==============================] - 76s 89ms/step - loss: 1.0533 - accuracy: 0.6155 - val_loss: 2.5696 - val_accuracy: 0.1945
Epoch 2/10
511/511 [==============================] - 43s 83ms/step - loss: 0.8465 - accuracy: 0.7043 - val_loss: 120.7705 - val_accuracy: 0.1775
Epoch 3/10
511/511 [==============================] - 43s 85ms/step - loss: 0.6890 - accuracy: 0.7612 - val_loss: 291.1301 - val_accuracy: 0.1136
Epoch 4/10
511/511 [==============================] - 42s 83ms/step - loss: 0.6055 - accuracy: 0.7926 - val_loss: 3.0075 - val_accuracy: 0.1645
Epoch 5/10
511/511 [==============================] - 43s 83ms/step - loss: 0.5592 - accuracy: 0.8

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


87910968/87910968 [==============================] - 5s 0us/step
Epoch 1/10
511/511 [==============================] - 91s 112ms/step - loss: 1.6709 - accuracy: 0.2979 - val_loss: 32174.3672 - val_accuracy: 0.1945
Epoch 2/10
511/511 [==============================] - 44s 86ms/step - loss: 1.5770 - accuracy: 0.3371 - val_loss: 4276.2007 - val_accuracy: 0.1632
Epoch 3/10
511/511 [==============================] - 44s 86ms/step - loss: 1.5805 - accuracy: 0.3198 - val_loss: 1381.1007 - val_accuracy: 0.1945
Epoch 4/10
511/511 [==============================] - 44s 86ms/step - loss: 1.5346 - accuracy: 0.3473 - val_loss: 238.5182 - val_accuracy: 0.1945
Epoch 5/10
511/511 [==============================] - 44s 85ms/step - loss: 1.4717 - accuracy: 0.3956 - val_loss: 54.7081 - val_accuracy: 0.1945
Epoch 6/10
511/511 [==============================] - 44s 85ms/step - loss: 1.4420 - accuracy: 0.4048 - val_loss: 2420.2573 - val_accuracy: 0.1945
Epoch 7/10
511/511 [==============================] - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/10
511/511 [==============================] - 96s 123ms/step - loss: 1.8044 - accuracy: 0.2091 - val_loss: 1.7869 - val_accuracy: 0.1945
Epoch 2/10
511/511 [==============================] - 54s 106ms/step - loss: 1.7824 - accuracy: 0.2163 - val_loss: 2.0353 - val_accuracy: 0.1919
Epoch 3/10
511/511 [==============================] - 54s 105ms/step - loss: 1.7807 - accuracy: 0.2163 - val_loss: 2.0419 - val_accuracy: 0.1919
Epoch 4/10
511/511 [==============================] - 54s 106ms/step - loss: 1.7801 - accuracy: 0.2163 - val_loss: 2.0429 - val_accuracy: 0.1919
Epoch 5/10
511/511 [==============================] - 54s 105ms/step - loss: 1.7800 - accuracy: 0.2163 - val_loss: 2.0415 - val_accuracy: 0.1919
Epoch 6/10
511/511 [==============================] - 54s 106ms/step - loss: 1.7799 - accuracy: 0.2163 - val_loss: 2.0431 - val_accuracy: 0.1919
Epoch 7/10
511/511 [==============================] - 54s 106ms/s

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/10
511/511 [==============================] - 99s 172ms/step - loss: 1.9680 - accuracy: 0.1934 - val_loss: 1.7897 - val_accuracy: 0.1580
Epoch 2/10
511/511 [==============================] - 67s 130ms/step - loss: 1.7892 - accuracy: 0.2137 - val_loss: 1.7873 - val_accuracy: 0.1945
Epoch 3/10
511/511 [==============================] - 66s 129ms/step - loss: 1.7807 - accuracy: 0.2163 - val_loss: 1.7875 - val_accuracy: 0.1945
Epoch 4/10
511/511 [==============================] - 66s 129ms/step - loss: 1.7804 - accuracy: 0.2163 - val_loss: 1.7875 - val_accuracy: 0.1945
Epoch 5/10
511/511 [==============================] - 66s 129ms/step - loss: 1.7804 - accuracy: 0.2163 - val_loss: 1.7873 - val_accuracy: 0.1945
Epoch 6/10
511/511 [==============================] - 66s 129ms/step - loss: 1.7809 - accuracy: 0.2163 - val_loss: 1.7880 - val_accuracy: 0.1945
Epoch 7/10
511/511 [==============================] - 66s 129ms/s

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
